# Using AWS Batch Processing to do preprocessing on congress data


#### HISTORY

* 4/29/21 mbod - try preproc for 97-114 congress data using AWS batch


### Trying launching multiple processes

* Before we resort to AWS Batch what happens if we try and launch multiple processes with SageMaker SDK?

In [6]:
import boto3
from sagemaker import get_execution_role
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput


In [7]:
region = boto3.session.Session().region_name 

account_id = boto3.client('sts').get_caller_identity().get('Account')
ecr_repository = 'sagemaker-processing-container'
tag = ':latest'
processing_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)
role = get_execution_role()


In [8]:
script_processor = ScriptProcessor(command=['python3'],
                image_uri=processing_repository_uri,
                role=role,
                instance_count=1,
                instance_type='ml.m5.2xlarge')


In [14]:
for chamber in [111,112,113,114]:
    cstr = '{:0>3}'.format(chamber)
    print('Scheduling', chamber)
    script_processor.run(code='scripts/congress_pre_process_withParty.py',
                     inputs=[ProcessingInput(
                        source=f's3://ascsagemaker/JMP_congressional_nmf/raw_data/speeches_{cstr}.txt',
                        destination='/opt/ml/processing/input/speeches'),
                            ProcessingInput(
                        source=f's3://ascsagemaker/JMP_congressional_nmf/raw_data/descr_{cstr}.txt',
                        destination='/opt/ml/processing/input/descr')
                            ],
                      outputs=[ProcessingOutput(
                        source='/opt/ml/processing/output',
                        destination='s3://ascsagemaker/JMP_congressional_nmf/House_bigrams')],
                      arguments=[cstr],
                     wait=False
                    )

Scheduling 111

Job Name:  sagemaker-processing-container-2021-08-27-15-21-25-148
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://ascsagemaker/JMP_congressional_nmf/raw_data/speeches_111.txt', 'LocalPath': '/opt/ml/processing/input/speeches', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-2', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://ascsagemaker/JMP_congressional_nmf/raw_data/descr_111.txt', 'LocalPath': '/opt/ml/processing/input/descr', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-064258348567/sagemaker-processing-container-2021-08-27-15-21-25-148/input/code/congress_pre_process_withParty.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 

In [6]:
script_processor.jobs[0].describe()

{'ProcessingInputs': [{'InputName': 'input-1',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://ascsagemaker/JMP_congressional_nmf/raw_data/speeches_110.txt',
    'LocalPath': '/opt/ml/processing/input/speeches',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}},
  {'InputName': 'input-2',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://ascsagemaker/JMP_congressional_nmf/raw_data/descr_110.txt',
    'LocalPath': '/opt/ml/processing/input/descr',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}},
  {'InputName': 'code',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-064258348567/sagemaker-processing-container-2021-05-10-12-48-35-529/input/code/congress_pre_process.py',
    'LocalPath': '/opt/ml/processing/input/code',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File'